<a href="https://colab.research.google.com/github/nazanin-zinouri/subclu/blob/main/chat_channel_ecosystem_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import scipy.stats as stats
from scipy import mean
from scipy.stats import t
from statsmodels.stats.contingency_tables import mcnemar
from sklearn.linear_model import LinearRegression
import datetime as dt
from datetime import date
import matplotlib.pyplot as plt
import numpy as np
import random
import pandas  as pd
import seaborn as sns
import gspread
from google.auth import default
from google.colab import auth, files
from google.colab import drive
# Authenticate
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
from google.cloud import bigquery
project_id = 'data-prod-165221'
client = bigquery.Client(project=project_id)

In [ ]:
revenue_query='''
SELECT
     DISTINCT pt,
     CASE WHEN pt<'2023-05-01' THEN 'pre' ELSE 'post' END AS time_period_glag,
     subreddit_name,
     revenue,
     impressions
FROM `reddit-employee-datasets.nazanin_zinouri.community_chat_impact_ad_impressions_and_revenue_20230620`
WHERE pt>='2023-03-21' AND pt<='2023-06-11'

'''

In [ ]:
ad_impresion_revenue_df = client.query(revenue_query).to_dataframe()

In [ ]:
ad_impresion_revenue_df.head()

,pt,time_period_glag,subreddit_name,revenue,impressions
0,2023-06-06 00:00:00+00:00,post,love,41.242229,7422
1,2023-06-02 00:00:00+00:00,post,love,14.033061,4637
2,2023-06-09 00:00:00+00:00,post,love,17.888934,6381
3,2023-05-23 00:00:00+00:00,post,love,18.900007,5936
4,2023-05-29 00:00:00+00:00,post,love,14.202471,5068


In [ ]:
##defining simple functions to calculate 1st and 3rd quantiles
def q1(x):
    return x.quantile(0.25)
def med(x):
    return x.quantile(0.50)
def q3(x):
    return x.quantile(0.75)

In [ ]:
## aggregate functions for groupby
f = {'impressions': ['min',med, 'mean', q1, q3,'max']}

In [ ]:
ad_impresion_revenue_df=ad_impresion_revenue_df[ad_impresion_revenue_df['subreddit_name']!='indian_hangouts']

In [ ]:
ad_impresion_revenue_df.groupby(['subreddit_name','time_period_glag']).agg(f)

impressions                         \
                                            min      med          mean   
subreddit_name     time_period_glag                                      
asksingapore       post                   70679  90006.5  87625.309524   
                   pre                    61220  74361.0  76886.390244   
belohorizonte      post                      22    520.0    512.190476   
                   pre                      276    620.0    629.780488   
britishcolumbia    post                    6346  19060.5  18929.119048   
                   pre                     9669  15661.0   15922.97561   
deathgrips         post                   11231  23168.0  30227.595238   
                   pre                     8622  13701.0  14408.487805   
hingeapp           post                   17285  35530.0  34985.285714   
                   pre                    27069  35314.0  35010.390244   
jamesbond          post                    2462   3590.0   3608.309524   
                   pre                     2337   3901.0   3879.804878   
love               post                    4163   5614.0   5688.142857   
                   pre                     3516   4938.0   5032.634146   
makenewfriendshere post                   37524  48627.5  48305.357143   
                   pre                    34663  42377.0  42387.902439   
r6proleague        post                    3468   7056.5   8843.309524   
                   pre                     6614  11662.0  11995.170732   
unimelb            post                    4964   8849.0  10544.880952   
                   pre                     4265   5705.0   5933.609756   
uraniumsqueeze     post                     632   1189.0    1315.52381   
                   pre                      676   1366.0   1372.390244   

                                                                 
                                           q1        q3     max  
subreddit_name     time_period_glag                              
asksingapore       post              81396.00  94427.00  107611  
                   pre               70511.00  82606.00   99678  
belohorizonte      post                398.50    605.50     905  
                   pre                 497.00    703.00    1024  
britishcolumbia    post              15995.00  22470.75   32371  
                   pre               13475.00  18371.00   24069  
deathgrips         post              18684.25  37042.75   98757  
                   pre               12434.00  15146.00   23209  
hingeapp           post              33439.00  37689.00   46020  
                   pre               32322.00  37128.00   41710  
jamesbond          post               3056.00   4160.75    5043  
                   pre                3502.00   4267.00    5637  
love               post               4876.50   6249.50    8345  
                   pre                4404.00   5377.00    7818  
makenewfriendshere post              46515.25  51219.00   55538  
                   pre               41371.00  44361.00   48769  
r6proleague        post               5263.00  10541.00   23417  
                   pre                9113.00  13737.00   20385  
unimelb            post               7816.50  10582.50   25985  
                   pre                5279.00   6497.00    9669  
uraniumsqueeze     post               1016.50   1531.00    3002  
                   pre                1055.00   1590.00    2577

In [ ]:
## aggregate functions for groupby
f2 = {'revenue': ['min',med, 'mean', q1, q3,'max']}

In [ ]:
ad_impresion_revenue_df.groupby(['subreddit_name','time_period_glag']).agg(f2)

revenue                          \
                                           min         med        mean   
subreddit_name     time_period_glag                                      
asksingapore       post              79.841180  107.330585  117.037989   
                   pre               67.651013  111.329172  113.471994   
belohorizonte      post               0.000000    0.128317    0.226647   
                   pre                0.007125    0.170775    0.220080   
britishcolumbia    post              15.521920   55.704386   56.116752   
                   pre               22.999286   41.478142   45.378269   
deathgrips         post              40.629475   93.440953  117.570834   
                   pre               27.292355   62.705461   66.357894   
hingeapp           post              27.027238   45.769602   45.328923   
                   pre               32.904516   55.135908   57.962140   
jamesbond          post               3.700496   10.711724   11.113244   
                   pre                2.577454   11.734111   13.081462   
love               post               8.192256   21.080269   21.881081   
                   pre                8.880833   16.575503   18.498829   
makenewfriendshere post              82.357833  149.347585  146.074880   
                   pre               64.544028  123.034970  125.340808   
r6proleague        post               9.796802   26.831086   28.711459   
                   pre               10.472221   35.915803   39.626611   
unimelb            post               8.329248   22.728621   26.068875   
                   pre                5.026741   12.175696   12.729528   
uraniumsqueeze     post               1.256604    4.987681    6.155930   
                   pre                0.619159    4.951474    5.635959   

                                                                         
                                             q1          q3         max  
subreddit_name     time_period_glag                                      
asksingapore       post               97.702595  132.229028  193.396080  
                   pre                97.275510  130.057591  170.023806  
belohorizonte      post                0.019672    0.291933    1.218631  
                   pre                 0.076881    0.295127    0.945834  
britishcolumbia    post               41.212468   65.583127  101.454329  
                   pre                37.080239   54.949549   73.313099  
deathgrips         post               77.734639  145.908837  358.438966  
                   pre                48.279281   82.529674  139.336166  
hingeapp           post               36.357488   54.385088   62.360496  
                   pre                46.361945   64.516911  110.111467  
jamesbond          post                8.159279   12.777044   23.752321  
                   pre                 8.734807   15.726109   32.576434  
love               post               15.603008   25.848943   49.422806  
                   pre                13.874904   22.043520   35.646197  
makenewfriendshere post              130.511230  162.522204  200.424222  
                   pre               110.682951  140.974768  186.549322  
r6proleague        post               17.072939   39.072950   72.980546  
                   pre                29.466183   51.433660   71.226874  
unimelb            post               16.243355   28.909105   65.497429  
                   pre                 9.066737   15.007299   29.498889  
uraniumsqueeze     post                2.596153    7.616798   21.418747  
                   pre                 2.381131    6.739666   16.674152

In [ ]:
sv_time_query='''
SELECT
     DISTINCT dt,
     CASE WHEN dt<'2023-05-01' THEN 'pre' ELSE 'post' END AS time_period_glag,
     subreddit_name,
     total_sessions,
     total_screenviews,
     totaltime_on_subreddit
FROM `reddit-employee-datasets.nazanin_zinouri.community_chat_impact_screenviews_and_timeonsite_20230620`
WHERE dt>='2023-03-21' AND dt<='2023-06-11'
'''

In [ ]:
sv_time_df = client.query(sv_time_query).to_dataframe()

In [ ]:
sv_time_df.head()

,dt,time_period_glag,subreddit_name,total_sessions,total_screenviews,totaltime_on_subreddit
0,2023-06-07,post,love,7694,21693,1626691
1,2023-03-24,pre,love,4726,15105,1317351
2,2023-03-22,pre,love,7035,18766,1381874
3,2023-06-09,post,love,7936,23226,1987757
4,2023-05-01,post,love,8072,22539,1803505


In [ ]:
sv_time_df=sv_time_df[sv_time_df['subreddit_name']!='indian_hangouts']

In [ ]:
## aggregate functions for groupby
f = {'total_sessions': ['min',med, 'mean', q1, q3,'max']}

In [ ]:
sv_time_df.groupby(['subreddit_name','time_period_glag']).agg(f)

total_sessions                           \
                                                min       med           mean   
subreddit_name      time_period_glag                                           
asksingapore        post                     103263  125454.0  123996.785714   
                    pre                       77230  101998.0  101322.219512   
belohorizonte       post                         39    1572.0    1746.904762   
                    pre                        1017    1961.0    2335.121951   
britishcolumbia     post                      23690   44391.0   46141.047619   
                    pre                       21480   36966.0   40025.243902   
deathgrips          post                      22071   31182.5   32054.333333   
                    pre                       16242   21402.0   21503.292683   
fallout76bowhunters post                        150     296.0     314.857143   
                    pre                         172     338.0     335.365854   
hingeapp            post                      28812   39196.0    39112.52381   
                    pre                       31628   38127.0   38384.341463   
jamesbond           post                       4661    5823.5    5862.190476   
                    pre                        4717    6978.0         7025.0   
love                post                       4944    6772.0    6921.380952   
                    pre                        4255    5920.0    6326.926829   
makenewfriendshere  post                      15927   19112.5   19285.619048   
                    pre                       18044   20134.0   20080.926829   
partnercommunities  post                         27     155.0     202.857143   
                    pre                          26     132.0     141.804878   
r6proleague         post                       4577    8070.0    8945.547619   
                    pre                        7256   11161.0   11335.829268   
unimelb             post                       4985   10449.5   11207.333333   
                    pre                        4426    6093.0    6214.585366   
uraniumsqueeze      post                        587    1353.0    1409.690476   
                    pre                         823    1617.0    1638.707317   

                                                                    
                                             q1         q3     max  
subreddit_name      time_period_glag                                
asksingapore        post              115824.75  131677.00  142391  
                    pre                90580.00  109412.00  127007  
belohorizonte       post                1254.00    2165.25    3689  
                    pre                 1727.00    2380.00   10661  
britishcolumbia     post               37830.25   52085.50   95392  
                    pre                31625.00   47732.00   69069  
deathgrips          post               27164.00   37003.25   44567  
                    pre                19635.00   22555.00   30305  
fallout76bowhunters post                 242.25     371.00     659  
                    pre                  297.00     356.00     680  
hingeapp            post               35384.00   43089.50   46023  
                    pre                35363.00   41008.00   48040  
jamesbond           post                5323.75    6296.25    7667  
                    pre                 6357.00    7980.00    8622  
love                post                5985.50    7485.75   11749  
                    pre                 5356.00    6858.00   11556  
makenewfriendshere  post               18528.75   20112.00   22616  
                    pre                19222.00   20717.00   22437  
partnercommunities  post                  99.50     236.50    1221  
                    pre                   72.00     210.00     306  
r6proleague         post                7024.75   10213.50   17413  
                    pre                10154.00   12838.00  

In [ ]:
f2 = {'total_screenviews': ['min',med, 'mean', q1, q3,'max']}

In [ ]:
sv_time_df.groupby(['subreddit_name','time_period_glag']).agg(f2)

total_screenviews            \
                                                   min       med   
subreddit_name      time_period_glag                               
asksingapore        post                        293144  385788.5   
                    pre                         264495  340033.0   
belohorizonte       post                            90    3506.5   
                    pre                           2403    4763.0   
britishcolumbia     post                         47691   87788.0   
                    pre                          43371   76015.0   
deathgrips          post                         66409  111804.5   
                    pre                          53233   68467.0   
fallout76bowhunters post                           331     983.5   
                    pre                            526    1170.0   
hingeapp            post                        113300  161877.0   
                    pre                         125522  157839.0   
jamesbond           post                         13290   16638.0   
                    pre                          13224   18895.0   
love                post                         14187   19997.5   
                    pre                          14820   18439.0   
makenewfriendshere  post                        111094  131148.0   
                    pre                         115902  127767.0   
partnercommunities  post                            63     519.0   
                    pre                             53     405.0   
r6proleague         post                         11763   24533.0   
                    pre                          24716   49843.0   
unimelb             post                         18289   36021.0   
                    pre                          15642   21628.0   
uraniumsqueeze      post                          2391    5677.0   
                    pre                           2838    6592.0   

                                                                           \
                                               mean         q1         q3   
subreddit_name      time_period_glag                                        
asksingapore        post              384949.380952  364613.25  409543.00   
                    pre               350224.585366  310464.00  379934.00   
belohorizonte       post                3779.047619    2830.25    4864.00   
                    pre                 5404.487805    4063.00    5624.00   
britishcolumbia     post               89940.904762   74509.00  102697.75   
                    pre                 80066.95122   63947.00   94544.00   
deathgrips          post               126599.97619   90964.50  141320.50   
                    pre                 73685.97561   63331.00   78827.00   
fallout76bowhunters post                1115.547619     809.25    1456.75   
                    pre                 1201.317073     987.00    1385.00   
hingeapp            post              159933.428571  144815.75  171978.00   
                    pre               158286.317073  149713.00  167833.00   
jamesbond           post               16637.380952   15308.75   17739.75   
                    pre                18900.585366   17360.00   20486.00   
love                post               19716.047619   17481.25   21649.75   
                    pre                18906.170732   16780.00   20306.00   
makenewfriendshere  post                   131206.0  126362.00  137977.25   
                    pre                128981.95122  125157.00  131997.00   
partnercommunities  post                 717.142857     326.75     894.75   
                    pre                  480.268293     190.00     737.00   
r6proleague         post               35784.785714   20021.25   44747.00   
                    pre                 53029.02439   42570.00   62568.00   
unimelb             post               40105.595238   28877.75   43981.00   
                    pre                22491.195122   19158.00